Источники:


1) [Using InterProScan like a pro (статья 2016)](https://medium.com/computer-says-no/using-interproscan-like-a-pro-ad18b8c3ccc0)

2) [файл iprscan5.py, но можно с помощью git (webservice-clients/python/iprscan5.py)](https://www.ebi.ac.uk/seqdb/confluence/display/JDSAT/InterProScan+5+Help+and+Documentation#InterProScan5HelpandDocumentation-RESTAPI) 

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.9 MB/s eta 0:00:00


In [2]:
import sys
import Bio
from Bio import SeqIO
from Bio.Blast import NCBIWWW

import urllib
import os
import re
import glob # для файлов в папке

from tqdm import tqdm
import shutil
from joblib import Parallel, delayed
import numpy as np

import requests
import json

from Bio.PDB import *
from Bio.PDB.PDBIO import PDBIO

import csv

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 1) Находим фибриллярные белки по ключевым словам

In [4]:
!git clone https://github.com/williamgilpin/pypdb.git

Cloning into 'pypdb'...
remote: Enumerating objects: 859, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 859 (delta 12), reused 18 (delta 11), pack-reused 831
Receiving objects: 100% (859/859), 557.69 KiB | 13.60 MiB/s, done.
Resolving deltas: 100% (471/471), done.


In [5]:
print("Текущая директория:", os.getcwd())
os.chdir("/content/pypdb")
print("Текущая директория изменилась:", os.getcwd())

Текущая директория: /content
Текущая директория изменилась: /content/pypdb


In [6]:
from pypdb.clients.search.search_client import perform_search_with_graph
from pypdb.clients.search.search_client import ReturnType
from pypdb.clients.search.search_client import QueryGroup, LogicalOperator
from pypdb.clients.search.operators import text_operators
from pypdb import *

In [7]:
fib_protein = Query("fibrous | fibrillar").search()
len(fib_protein)

3663

In [ ]:
file1 = open("fibrous_fibrillar.txt", "w")
for i in fib_protein:
  file1.writelines(f'{str(i)}\n')
file1.close()

#2) Скачиваю fasta файлы фибриллярных белков

In [8]:
%%bash 
mkdir /content/FIB_PROTEIN
mkdir /content/FIB_PROTEIN/fasta

In [9]:
def download_fasta(fastacode, datadir, downloadurl="https://www.rcsb.org/fasta/entry/"):

    url = downloadurl + fastacode
    outfnm = os.path.join(datadir, fastacode)
    try:
        urllib.request.urlretrieve(url, outfnm)
        return outfnm
    except Exception as err:
        print(str(err), file=sys.stderr)
        return str(err) #None

# бывают проблемы с фаста файлами, не всегда они есть, поэтому следующая функция
def only_fasta(i_results):
  if download_fasta(i_results, '/content/FIB_PROTEIN/fasta/') == 'HTTP Error 404: Not Found':
    print(f'Нет fasta файла для белка {i_results}')
  else:
    download_fasta(i_results, '/content/FIB_PROTEIN/fasta/')
# for i in fib_protein:
only_fasta('1U5M')

In [ ]:
print(f"количество белков = {len(fib_protein)}")

количество белков = 3663


In [ ]:
%%bash
echo "количество файлов = $(ls /content/FIB_PROTEIN/fasta | wc -l)"

количество файлов = 3663


# 3) Pазделение на домены 

### 3.1) создание единого fasta файла (так нужно для дальнейшего поиска, для Pfam)

In [11]:
%%bash 
mkdir /content/FIB_PROTEIN/fasta_one

In [12]:
path = '/content/FIB_PROTEIN/fasta/*'
for file in glob.glob(path):
  with open(file, 'r') as pdb_file:
    fasta_file=''
    for record in SeqIO.parse(pdb_file, 'fasta'):
      fasta_file+=record.seq
    # print(pdb_file.name)
    # print(pdb_file.name[27:])
    # print(f'result={fasta_file}')
    
    with open(f"/content/FIB_PROTEIN/fasta_one/{pdb_file.name[27:]}_new.fasta", "w") as external_file:  
      print('>' + pdb_file.name[27:], file=external_file)
      print(fasta_file, file=external_file)
    external_file.close()

### 3.2) Pfam

In [13]:
print("Текущая директория:", os.getcwd())
os.chdir("/content/FIB_PROTEIN")
print("Текущая директория изменилась:", os.getcwd())

Текущая директория: /content/pypdb
Текущая директория изменилась: /content/FIB_PROTEIN


In [14]:
%%bash 
mkdir /content/FIB_PROTEIN/InterPro
mkdir /content/FIB_PROTEIN/InterPro/tsv_result

In [15]:
%%bash
#!/bin/bash
for file in /content/FIB_PROTEIN/fasta_one/*
do
	echo file=$file
	filename=`basename ${file%%.*}`
  echo filename=$filename
	name_protein=$(echo $filename | cut -c 1-4 )
  echo protein=$name_protein
done

file=/content/FIB_PROTEIN/fasta_one/1U5M_new.fasta
filename=1U5M_new
protein=1U5M


In [ ]:
# !rm -r /content/FIB_PROTEIN/Pfam/

In [16]:
!cd /content
!git clone https://github.com/ebi-wp/webservice-clients.git
!cd webservice-clients

Cloning into 'webservice-clients'...
remote: Enumerating objects: 20567, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (242/242), done.
remote: Total 20567 (delta 419), reused 302 (delta 245), pack-reused 20079
Receiving objects: 100% (20567/20567), 240.50 MiB | 24.27 MiB/s, done.
Resolving deltas: 100% (15265/15265), done.


In [17]:
!pip install xmltramp2==3.0.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for xmltramp2: filename=xmltramp2-3.0.10-py3-none-any.whl size=7305 sha256=2e2def5725b5ebacc4642681bcced60b6475f5d507af9d3a15cc574d4fc8202f
  Stored in directory: /root/.cache/pip/wheels/e0/a2/86/df51835a4089613017a5029b4125eee71d9c7770a7916d78f5
Successfully built xmltramp2


In [ ]:
%%bash
for j in $(find `pwd` -type f -name "1A31_new.fasta")
do
    echo "Iteration: $i; File: $j"
    filename1=`basename ${j%%.*}`
    filename2=$(basename "$j")
    echo $filename1, $filename2
done

Iteration: ; File: /content/FIB_PROTEIN/fasta_one/1A31_new.fasta
1A31_new, 1A31_new.fasta


In [18]:
%%bash
#!/bin/bash
i=1
waitevery=30

filename=/content/FIB_PROTEIN/fasta_one/1A31_new.fasta

for j in $(find `pwd` -type f -name "*_new.fasta")
do
    echo "Iteration: $i; File: $j"
    filename=`basename ${j%%.*}`
    

    echo $filename
    python3 /content/FIB_PROTEIN/webservice-clients/python/iprscan5.py \
        --goterms \
        --pathways \
        --email=e.mazinova@g.nsu.ru \
        --outfile=/content/FIB_PROTEIN/InterPro/tsv_result/${filename} \
        --outformat=tsv \
        --quiet \
        $j & (( i++%waitevery==0 )) && wait
done

Iteration: 1; File: /content/FIB_PROTEIN/fasta_one/1U5M_new.fasta
1U5M_new
iprscan5-R20230125-062628-0335-95058856-p1m


CalledProcessError: ignored

In [22]:
find_protein_after_domains=[]
for file in glob.glob("/home/elzara/FIB_PROTEIN/InterPro/tsv_result/1*_new.tsv.tsv"):
  print(f"file = {file}")
  tsv_file = csv.reader(file, delimiter="\t")
  print(tsv_file)
  for line in tsv_file:
    print(line)
    if (line[3] == "Pfam"):
      find_word = re.findall('(fibr[a-z]+)|(fiber)', line[5])
      if len(find_word)!=0:
        print(f"FIND FIBROUS PROTEIN ===> {line[0]}")
        find_protein_after_domains.append(line[0])
    # print(f"program = {line[3]} => name = {line[5]}   /////  start={line[6]}, end={line[7]}")

In [26]:
find_protein_after_domains=[]
for file in glob.glob("/home/elzara/FIB_PROTEIN/InterPro/tsv_result/1*_new.tsv.tsv"):
        with open(file) as protein:
                print(f"protein = {protein}")
                tsv_file = csv.reader(protein, delimiter="\t")
                for line in tsv_file:
                        print(f"line of tsv file:\n{line}")
                        if (line[3] == "Pfam"):
                                find_word = re.findall('(fibr[a-z]+)|(fiber)', line[5])
                                if len(find_word)!=0:
                                        print(f"FIND FIBROUS PROTEIN ===> {line[0]}")
                                        find_protein_after_domains.append(line[0])

In [25]:
import csv
with open("/content/FIB_PROTEIN/InterPro/tsv_result/1U5M_new.tsv.tsv") as file:
  tsv_file = csv.reader(file, delimiter="\t")
  for line in tsv_file:
    print(f"program = {line[3]} => name = {line[5]}   /////  start={line[6]}, end={line[7]}") 

program = Pfam => name = von Willebrand factor type C domain   /////  start=10, end=65
program = Gene3D => name = Complement Module, domain 1   /////  start=9, end=44
program = SMART => name = vwc   /////  start=10, end=65
program = SUPERFAMILY => name = FnI-like domain   /////  start=8, end=67
program = ProSiteProfiles => name = VWFC domain profile.   /////  start=8, end=66
program = ProSitePatterns => name = VWFC domain signature.   /////  start=28, end=65
program = FunFam => name = Collagen, type I, alpha 1   /////  start=9, end=44


In [ ]:
import csv
with open("/content/1A31_new.tsv.tsv") as file:
  tsv_file = csv.reader(file, delimiter="\t")
  for line in tsv_file:
    print(f"program = {line[3]} => name = {line[5]}   /////  start={line[6]}, end={line[7]}")      

program = PANTHER => name = DNA TOPOISOMERASE I   /////  start=45, end=635
program = Phobius => name = Signal peptide region   /////  start=1, end=26
program = Phobius => name = Hydrophobic region of a signal peptide.   /////  start=3, end=14
program = CDD => name = Topo_IB_C   /////  start=308, end=508
program = Phobius => name = N-terminal region of a signal peptide.   /////  start=1, end=2
program = FunFam => name = DNA topoisomerase I   /////  start=190, end=299
program = MobiDBLite => name = consensus disorder prediction   /////  start=45, end=66
program = Coils => name = Coil   /////  start=514, end=548
program = Gene3D => name = Topoisomerase I; Chain A, domain 3   /////  start=302, end=456
program = SignalP_GRAM_POSITIVE => name = SignalP-TM   /////  start=1, end=26
program = Phobius => name = C-terminal region of a signal peptide.   /////  start=15, end=26
program = Gene3D => name = -   /////  start=457, end=635
program = SignalP_GRAM_NEGATIVE => name = SignalP-noTM   /////  s

In [ ]:

%%bash
file=/content/FIB_PROTEIN/fasta_one/1A31_new.fasta
curl -LH 'Expect:' -F seq=$file -F output=xml 'https://www.ebi.ac.uk/interpro/search/sequence/' > /content/res_$name_protein.xml

In [ ]:
find_protein_after_domains=[]
with open('out_of_fibrous.txt', 'w') as f:
  for file in glob.glob("/home/elzara/FIB_PROTEIN/InterPro/tsv_result/*_new.tsv.tsv"):
    with open(file) as protein:
      # print(f"protein = {protein}")
      tsv_file = csv.reader(protein, delimiter="\t")
      for line in tsv_file:
        # print(f"line of tsv file:\n{line}")
        #if (line[3] == "Pfam"):
        #print(f"prot = {line[0]}")
        #print(f"name = {line[5]}")
        find_word = re.findall('([Ff]ibr[a-z]+)|([Ff]iber)|([Cc]ollagen)|([Kk]eratin)|([Ee]lastin)', lin>      if len(find_word)!=0:
          print(f"{line[0][4:]}")
          print(f"find pattern = {find_word}")
          print(f"name = {line[5]}")
          find_protein_after_domains.append(line[0][4:])